In [1]:
from pathlib import Path
import re
import pandas as pd
from IPython.display import Audio, display
from pydub import AudioSegment
import numpy as np
from textwrap import wrap
from time import sleep

In [2]:
AUDIODIR = "/media/pers/elements/npscpluss_audio/segmented"
TEST_AUDIO_FILE = "test_clip.wav"
ANNOTATED_DATASET_PATH = "data/annotated_dataset.csv"

In [3]:
pattern = re.compile(r"^[sS]tortinget-(\d{4}).*")

def prepare_segment(row, audiofolder):
    audio = list(row.audiofilename)[0] + ".mp3"
    year = pattern.match(audio).group(1)
    segment = AudioSegment.from_mp3(Path(AUDIODIR) / year / audio)
    segment.export(TEST_AUDIO_FILE, format='wav')
    print(f"File: {audio}\n")
    print(f"Language: {list(row.language)[0]}")
    print("corpus:\n" + '\n'.join(wrap(list(row.proceedings_text)[0], width=100)) + "\n")
    print("asr:\n" + '\n'.join(wrap(list(row.transcription_text)[0], width=100)) + "\n")


In [4]:
annotation_df = pd.read_csv(ANNOTATED_DATASET_PATH)

In [5]:
def get_annotation(row):
    row.missing_start_word = (input("Missing start word?") != "") 
    row.missing_end_word = (input("Missing end word?") != "")
    row.wrong_match = (input("Wrong match?") != "")
    row.wrong_language = (input("Wrong language?") != "")
    row.low_quality = (input("Low quality?") != "")
    return row

In [6]:
def annotate_data(filename, audiofolder):
    df = pd.read_csv(filename)
    updated_cells = [
        "missing_start_word",
        "missing_end_word",
        "wrong_match",
        "wrong_language",
        "low_quality",
    ]
    subset_df = df[df.missing_start_word.isna()]
    random_row = subset_df.sample()
    prepare_segment(random_row, audiofolder)
    display(Audio(TEST_AUDIO_FILE, autoplay=True))
    random_row = get_annotation(random_row)
    df.loc[random_row.index] = random_row
    df.to_csv(filename, index=False)


    

In [7]:
annotate_data("data/annotated_dataset.csv", AUDIODIR)

File: Stortinget-20170308-095504_2895800_2909400.mp3

Language: bm
corpus:
Det siste jeg hørte fra det møtet som var mellom Nav og Thor-Egil sist uke, var at nå var Thor-Egil
fornøyd med den hjelpen han hadde fått. Det er viktig at Nav i møte med brukere som trenger hjelp,
gjør godt arbeid, viser godt skjønn

asr:
eee og det siste jeg hørte fra det møtet som var mellom nav og torregel siste uke var at nå også
torregel var fornøyd med den hjelpen hadde hadde fått og det er viktig at at nav i møte med brukere
som trenger hjelp eee gjør godt arbeid viser godt skjønn

